In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import os
from PIL import Image
import numpy as np
import torch
from torch.utils.data import Dataset
from torchvision import transforms as T
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
class PetDatasetWrapper(Dataset):
    def __init__(self, root, split='trainval', augmentations=None, download=True):
        # 1. 在内部加载 torchvision 的原始数据集
        self.base_dataset = torchvision.datasets.OxfordIIITPet(
            root=root,
            split=split,
            target_types='segmentation',
            download=download
        )
        self.augmentations = augmentations

    def __len__(self):
        return len(self.base_dataset)

    def __getitem__(self, idx):

        image, mask = self.base_dataset[idx]
        
        image = np.array(image)
        mask = np.array(mask)
        
        # 2. 对掩码进行重映射 (这是核心步骤!)
        # 原: 1=Pet, 2=Background, 3=Border
        # 新: 1=Pet, 0=Background, 255=Ignore
        mask[mask == 2] = 0  # 背景: 2 -> 0
        mask[mask == 3] = 255 # 边界: 3 -> 255 (忽略)
        # 宠物类别 1 保持不变

        # 3. 应用 Albumentations 增强
        if self.augmentations:
            augmented = self.augmentations(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']
            
        return image, mask



In [3]:
IMAGE_SIZE = (256, 256)
BATCH_SIZE = 8
DATA_ROOT = '/kaggle/working/input'

# 数据增强
train_augs = A.Compose([
    # 首先将图像缩放到一个稍大的尺寸，再进行随机裁剪
    A.Resize(int(IMAGE_SIZE[0] * 1.25), int(IMAGE_SIZE[1] * 1.25)),
    A.RandomCrop(width=IMAGE_SIZE[0], height=IMAGE_SIZE[1]),
    
    # 几何变换
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=35, p=0.5),
    
    # 像素/颜色变换 (只会作用于 image)
    A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2, p=0.5),
    A.Blur(blur_limit=3, p=0.3),

    # 归一化和转换为Tensor
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])

val_augs = A.Compose([
    A.Resize(width=IMAGE_SIZE[0], height=IMAGE_SIZE[1]),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])



# --- 关键变更：实例化新的数据集和加载器 ---
# OxfordIIITPet 默认分为 'trainval' 和 'test'
train_dataset = PetDatasetWrapper(root=DATA_ROOT, split='trainval', augmentations=train_augs,download=True)
val_dataset = PetDatasetWrapper(root=DATA_ROOT, split='test', augmentations=val_augs,download=True)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

print(f"训练集样本数: {len(train_dataset)}")
print(f"验证集样本数: {len(val_dataset)}")
# 检查一个样本的输出
img, msk = train_dataset[0]
print(f"图像张量尺寸: {img.shape}, 类型: {img.dtype}")
print(f"掩码张量尺寸: {msk.shape}, 类型: {msk.dtype}")
print(f"掩码中的唯一值: {torch.unique(msk)}") # 应该会看到 0, 1, 和可能的 255


100%|██████████| 792M/792M [00:28<00:00, 28.0MB/s]
100%|██████████| 19.2M/19.2M [00:02<00:00, 8.69MB/s]


训练集样本数: 3680
验证集样本数: 3669
图像张量尺寸: torch.Size([3, 256, 256]), 类型: torch.float32
掩码张量尺寸: torch.Size([256, 256]), 类型: torch.uint8
掩码中的唯一值: tensor([  0,   1, 255], dtype=torch.uint8)


In [4]:
# 构建网络模型
import torch.nn as nn
import torch.nn.functional as F


class FCN(nn.Module):
    def __init__(self, num_classes=21):
        super().__init__()
        
        # 256 -> 128 -> 64 -> 32 -> 16 -> 8
        
        # Block 1: 256 -> 128
        self.conv_block1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        # Block 2: 128 -> 64
        self.conv_block2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        # Block 3: 64 -> 32
        self.conv_block3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        # Block 4: 32 -> 16
        self.conv_block4 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        # Block 5: 16 -> 8
        self.conv_block5 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        # ==================== 分类器 (1x1 Conv) ====================
        self.classifier = nn.Conv2d(512, num_classes, kernel_size=1)
        
        # ==================== 上采样路径 ====================
        # 我们需要从 8x8 恢复到 256x256，需要上采样 32 倍
        self.upsample = nn.ConvTranspose2d(num_classes, num_classes, kernel_size=64, stride=32, padding=16) # (kernel_size - stride) / 2
    def forward(self, x):
        # 记录原始尺寸，以防万一
        input_size = x.shape[-2:]
        
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = self.conv_block3(x)
        x = self.conv_block4(x)
        x = self.conv_block5(x)
        
        x = self.classifier(x)
        
        x = self.upsample(x)
        
        # 如果因为计算精度导致尺寸差一点点，可以用插值来修正
        x = F.interpolate(x, size=input_size, mode='bilinear', align_corners=False)
        
        return x

import torch.nn as nn
import torch.nn.functional as F

# class FCN8s(nn.Module):
#     def __init__(self, num_classes=21):
#         super().__init__()

#         # --- 编码器路径 (和之前一样) ---
#         # Block 1: 256 -> 128
#         self.conv_block1 = nn.Sequential(...) # 保持不变
#         # Block 2: 128 -> 64
#         self.conv_block2 = nn.Sequential(...) # 保持不变
#         # Block 3: 64 -> 32
#         self.conv_block3 = nn.Sequential(...) # 保持不变
#         # Block 4: 32 -> 16
#         self.conv_block4 = nn.Sequential(...) # 保持不变
#         # Block 5: 16 -> 8
#         self.conv_block5 = nn.Sequential(...) # 保持不变

#         # --- 分类器 (和之前一样) ---
#         self.classifier = nn.Conv2d(512, num_classes, kernel_size=1)

#         # ==================== 【核心修改】解码器路径 ====================
#         # 1. 对编码器第4个池化层的输出进行1x1卷积
#         self.score_pool4 = nn.Conv2d(512, num_classes, kernel_size=1)
        
#         # 2. 对编码器第3个池化层的输出进行1x1卷积
#         self.score_pool3 = nn.Conv2d(256, num_classes, kernel_size=1)

#         # 3. 上采样层
#         # 第一次上采样 (2倍)
#         self.upsample_2x_1 = nn.ConvTranspose2d(num_classes, num_classes, kernel_size=4, stride=2, padding=1, bias=False)
#         # 第二次上采样 (2倍)
#         self.upsample_2x_2 = nn.ConvTranspose2d(num_classes, num_classes, kernel_size=4, stride=2, padding=1, bias=False)
#         # 第三次上采样 (8倍)
#         self.upsample_8x = nn.ConvTranspose2d(num_classes, num_classes, kernel_size=16, stride=8, padding=4, bias=False)

#     def forward(self, x):
#         input_size = x.shape[-2:]

#         # --- 编码器前向传播，并保存中间结果 ---
#         h = self.conv_block1(x)
#         h = self.conv_block2(h)
#         pool3_out = self.conv_block3(h)   # 保存第3个池化后的输出 (H/8)
#         pool4_out = self.conv_block4(pool3_out) # 保存第4个池化后的输出 (H/16)
#         h = self.conv_block5(pool4_out) # (H/32)

#         # --- 解码器前向传播 ---
#         # 1. 对最深的特征图进行分类和2倍上采样
#         h = self.classifier(h)
#         h = self.upsample_2x_1(h) # -> H/16
        
#         # 2. 融合pool4的特征 (第一次跳跃连接)
#         score4 = self.score_pool4(pool4_out)
#         # 裁剪score4以匹配上采样后的h的尺寸 (FCN论文中的细节)
#         c = (score4.shape[3] - h.shape[3]) // 2
#         score4 = score4[:, :, c:c + h.shape[2], c:c + h.shape[3]]
#         h = h + score4 # 融合
        
#         # 3. 再次2倍上采样
#         h = self.upsample_2x_2(h) # -> H/8
        
#         # 4. 融合pool3的特征 (第二次跳跃连接)
#         score3 = self.score_pool3(pool3_out)
#         c = (score3.shape[3] - h.shape[3]) // 2
#         score3 = score3[:, :, c:c + h.shape[2], c:c + h.shape[3]]
#         h = h + score3 # 融合

#         # 5. 最后进行8倍上采样，恢复到原始尺寸
#         h = self.upsample_8x(h) # -> H
        
#         # 裁剪到原始输入尺寸
#         c = (h.shape[3] - input_size[1]) // 2
#         h = h[:, :, c:c + input_size[0], c:c + input_size[1]]
        
#         return h



net = FCN(num_classes=2)
net = net.to(device)

In [5]:
import torch.optim as optim
# 损失函数
loss_function = nn.CrossEntropyLoss(ignore_index=255)  # PASCAL VOC中255是边界或忽略区域，使用 ignore_index=255 来忽略PASCAL VOC中的边界像素，边界像素(255)在计算损失时会被忽略
# 优化器
optimizer = optim.Adam(net.parameters(), lr=0.001)
# 学习率调度器
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 
    mode='min',      # 监控的指标越小越好 (我们的目标是降低loss)
    factor=0.2,      # 当指标不再改善时，学习率乘以这个因子 (lr = lr * 0.2)
    patience=5,      # 容忍指标连续5个epoch不改善，才降低学习率
    verbose=True,    # 当学习率更新时，在控制台打印一条消息
    min_lr=1e-6      # 学习率的下限，防止降得太低
)

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [6]:
from torch.utils.tensorboard import SummaryWriter
# --- 1. 初始化 ---
epochs = 200
writer = SummaryWriter("/kaggle/working/logs/pet")
best_val_loss = float('inf') # 用于保存最佳模型的变量
MODEL_SAVE_PATH = '/kaggle/working/fcn_pet_best.pth'

print('Starting Training...')
# --- 2. 主训练循环 ---
for epoch in range(epochs): 
    print(f"--- Epoch {epoch + 1}/{epochs} ---")
    
    # --- 训练阶段 ---
    net.train() # 对于包含 Dropout 层或 Batch Normalization 层的模型需要调用 model.train() 和 model.eval() 
    train_loss = 0.0
    train_correct_pixels = 0
    train_total_pixels = 0
    
    # 使用tqdm包装训练数据加载器
    train_progress_bar = tqdm(train_loader, desc="Training", colour="green")
    
    for inputs, masks in train_progress_bar:
        inputs = inputs.to(device)
        masks  = masks.to(device, dtype=torch.long) # 确保掩码是长整型张量，因为 CrossEntropyLoss 需要标签是整数类型
        # 清除梯度->向前传播->计算损失->反向传播->优化参数
        optimizer.zero_grad() # zero_grad() 必须在 loss.backward()前. 在当前训练批次开始时，首先清零所有参数的梯度。
        outputs = net(inputs)
        loss = loss_function(outputs, masks)
        loss.backward()
        optimizer.step()

         # 累积损失
        train_loss += loss.item()

        # 计算像素准确率
        with torch.no_grad():
            preds = torch.argmax(outputs, dim=1)
            # 只计算非忽略区域的像素
            valid_pixels = (masks != 255)
            train_correct_pixels += ((preds == masks) & valid_pixels).sum().item()
            train_total_pixels += valid_pixels.sum().item()
        
        # 使用 set_postfix 在进度条上动态显示当前批次的损失
        train_progress_bar.set_postfix(loss=f"{loss.item():.4f}")

    # --- 验证阶段 ---
    net.eval()
    val_loss = 0.0
    val_correct_pixels = 0
    val_total_pixels = 0

    # 使用tqdm包装验证数据加载器
    val_progress_bar = tqdm(val_loader, desc="Validation", colour="red")
    
    with torch.no_grad(): # 测试过程中不计算梯度
        for inputs, masks in val_progress_bar:
            inputs = inputs.to(device)
            masks = masks.to(device, dtype=torch.long)
            outputs = net(inputs)
            loss = loss_function(outputs, masks)

            val_loss += loss.item()

            preds = torch.argmax(outputs, dim=1)
            valid_pixels = (masks != 255)
            val_correct_pixels += ((preds == masks) & valid_pixels).sum().item()
            val_total_pixels += valid_pixels.sum().item()

            val_progress_bar.set_postfix(loss=f"{loss.item():.4f}")


    # --- 3. 计算并记录每个Epoch的指标 ---
    # 计算平均损失和准确率
    avg_train_loss = train_loss / len(train_loader)
    train_pixel_acc = (train_correct_pixels / train_total_pixels) * 100 if train_total_pixels > 0 else 0
    
    avg_val_loss = val_loss / len(val_loader)
    val_pixel_acc = (val_correct_pixels / val_total_pixels) * 100 if val_total_pixels > 0 else 0
    # 打印本轮的最终结果
    print(f"\nEpoch {epoch + 1} Summary:")
    print(f"  Train Loss: {avg_train_loss:.4f}, Train Pixel Accuracy: {train_pixel_acc:.2f}%")
    print(f"  Validation Loss: {avg_val_loss:.4f}, Validation Pixel Accuracy: {val_pixel_acc:.2f}%")
    # 记录到TensorBoard
    writer.add_scalar('Loss/train', avg_train_loss, epoch)
    writer.add_scalar('Accuracy/train', train_pixel_acc, epoch)
    writer.add_scalar('Loss/validation', avg_val_loss, epoch)
    writer.add_scalar('Accuracy/validation', val_pixel_acc, epoch)
    writer.add_scalar('Learning_Rate', optimizer.param_groups[0]['lr'], epoch)
    
    # 更新学习率，并将验证集损失传入
    scheduler.step(avg_val_loss)
    print(f"  Current learning rate: {optimizer.param_groups[0]['lr']:.6f}")

    # --- 4. 保存最佳模型 ---
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(net.state_dict(), MODEL_SAVE_PATH)
        print(f"  ---> New best model saved to {MODEL_SAVE_PATH} (Validation Loss: {best_val_loss:.4f})")
    
    print("-" * 30)

print('Finished Training')
print(f"Best model saved at {MODEL_SAVE_PATH} with validation loss: {best_val_loss:.4f}")
writer.close()


2025-07-22 08:10:24.406722: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753171824.591055      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753171824.642758      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Starting Training...
--- Epoch 1/200 ---


Validation: 100%|██████████| 459/459 [00:29<00:00, 15.75it/s, loss=0.5262]



Epoch 1 Summary:
  Train Loss: 0.6233, Train Pixel Accuracy: 64.57%
  Validation Loss: 0.5248, Validation Pixel Accuracy: 74.12%
  Current learning rate: 0.001000
  ---> New best model saved to /kaggle/working/fcn_pet_best.pth (Validation Loss: 0.5248)
------------------------------
--- Epoch 2/200 ---


Validation: 100%|██████████| 459/459 [00:29<00:00, 15.80it/s, loss=0.5303]


...


Epoch 198 Summary:
  Train Loss: 0.1358, Train Pixel Accuracy: 94.68%
  Validation Loss: 0.1304, Validation Pixel Accuracy: 95.26%
  Current learning rate: 0.000001
------------------------------
--- Epoch 199/200 ---


Validation: 100%|██████████| 459/459 [00:29<00:00, 15.38it/s, loss=0.1428]



Epoch 199 Summary:
  Train Loss: 0.1361, Train Pixel Accuracy: 94.63%
  Validation Loss: 0.1306, Validation Pixel Accuracy: 95.25%
  Current learning rate: 0.000001
------------------------------
--- Epoch 200/200 ---


Validation: 100%|██████████| 459/459 [00:30<00:00, 15.27it/s, loss=0.1425]


Epoch 200 Summary:
  Train Loss: 0.1352, Train Pixel Accuracy: 94.69%
  Validation Loss: 0.1306, Validation Pixel Accuracy: 95.25%
  Current learning rate: 0.000001
------------------------------
Finished Training
Best model saved at /kaggle/working/fcn_pet_best.pth with validation loss: 0.1280


In [7]:

import matplotlib.pyplot as plt


MODEL_PATH = '/kaggle/working/fcn_pet_best.pth' # 你训练好的模型路径
OUTPUT_DIR = '/kaggle/working/outputs/results/'

NUM_CLASSES = 2
IMAGE_SIZE = (256, 256)
NUM_VISUALIZATIONS = 10

os.makedirs(OUTPUT_DIR, exist_ok=True)

# --- 1. 颜色映射---
PET_COLORMAP = np.array([
    [0, 0, 0],       # 类别 0: 背景 (黑色)
    [128, 0, 0],     # 类别 1: 宠物 (红色)
], dtype=np.uint8)

def mask_to_color(mask, colormap):
    # 创建一个三通道的彩色掩码
    color_mask = np.zeros((mask.shape[0], mask.shape[1], 3), dtype=np.uint8)
    # 遍历颜色映射表中的每一种颜色
    for class_idx, color in enumerate(colormap):
        # 将掩码中所有等于当前类别索引的像素，都赋值为对应的颜色
        color_mask[mask == class_idx] = color
    return color_mask



# 定义数据变换
val_augs = A.Compose([
    A.Resize(width=IMAGE_SIZE[0], height=IMAGE_SIZE[1]),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])
# 加载模型
model = FCN(num_classes=NUM_CLASSES).to(device)
model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model.eval()
print("模型加载成功！")
# --- 5. 关键变更：准备数据集的方式 ---
# 我们需要两个数据集实例：
# a) 一个应用了变换，用于输入模型
test_dataset_transformed = PetDatasetWrapper(root=DATA_ROOT, split='test', augmentations=val_augs, download=False)
# b) 一个不应用任何变换，用于获取原始图像和ID
test_dataset_original = torchvision.datasets.OxfordIIITPet(root=DATA_ROOT, split='test', download=False)
print(f"开始在 'test' 集上进行预测和可视化...")
with torch.no_grad():
    for i in range(min(NUM_VISUALIZATIONS, len(test_dataset_transformed))):
        # --- 6. 关键变更：获取数据的方式 ---
        
        # a) 从转换后的数据集中获取模型输入和真值掩码
        image_tensor, gt_mask = test_dataset_transformed[i]
        
        # b) 从原始数据集中获取原始图像和文件名ID
        original_image, _ = test_dataset_original[i] # 这是原始的PIL Image
        # torchvision.datasets.OxfordIIITPet 的文件名存储在 _images 属性中
        image_id = os.path.splitext(os.path.basename(test_dataset_original._images[i]))[0]
        # -------------------------------------------------
        
        # 模型需要一个batch维度，所以我们用unsqueeze(0)增加一个维度
        image_tensor = image_tensor.unsqueeze(0).to(device)
        
        # 模型预测
        output = model(image_tensor)
        pred_mask = torch.argmax(output, dim=1)
        
        # 转为Numpy
        pred_mask_np = pred_mask.cpu().squeeze(0).numpy()
        gt_mask_np = gt_mask.cpu().numpy() # gt_mask 已经是tensor了，直接转numpy
        
        # 将忽略区域(255)处理一下，方便可视化，统一设为背景类别0
        gt_mask_np[gt_mask_np == 255] = 0 
        
        # --- 7. 关键变更：使用新的颜色映射表 ---
        pred_mask_color = mask_to_color(pred_mask_np, PET_COLORMAP)
        gt_mask_color = mask_to_color(gt_mask_np, PET_COLORMAP)
        
        # 绘图
        fig, axes = plt.subplots(1, 3, figsize=(18, 6))
        
        # 显示原始图像 (需要resize以匹配输出尺寸)
        axes[0].imshow(original_image.resize(IMAGE_SIZE))
        axes[0].set_title(f'Original Image: {image_id}')
        axes[0].axis('off')
        
        axes[1].imshow(gt_mask_color)
        axes[1].set_title('Ground Truth Mask')
        axes[1].axis('off')
        
        axes[2].imshow(pred_mask_color)
        axes[2].set_title('Predicted Mask')
        axes[2].axis('off')
        
        plt.tight_layout()
        
        save_path = os.path.join(OUTPUT_DIR, f'{image_id}_pred.png')
        plt.savefig(save_path)
        plt.close(fig) # 关闭图形，防止在Jupyter中重复显示
        print(f"已保存可视化结果: {save_path}")
print(f"\n可视化完成！结果已保存在 '{OUTPUT_DIR}' 文件夹中。")

模型加载成功！
开始在 'test' 集上进行预测和可视化...
已保存可视化结果: /kaggle/working/outputs/results/Abyssinian_201_pred.png
已保存可视化结果: /kaggle/working/outputs/results/Abyssinian_202_pred.png
已保存可视化结果: /kaggle/working/outputs/results/Abyssinian_204_pred.png
已保存可视化结果: /kaggle/working/outputs/results/Abyssinian_205_pred.png
已保存可视化结果: /kaggle/working/outputs/results/Abyssinian_206_pred.png
已保存可视化结果: /kaggle/working/outputs/results/Abyssinian_207_pred.png
已保存可视化结果: /kaggle/working/outputs/results/Abyssinian_20_pred.png
已保存可视化结果: /kaggle/working/outputs/results/Abyssinian_210_pred.png
已保存可视化结果: /kaggle/working/outputs/results/Abyssinian_212_pred.png
已保存可视化结果: /kaggle/working/outputs/results/Abyssinian_213_pred.png

可视化完成！结果已保存在 '/kaggle/working/outputs/results/' 文件夹中。
